# **ASOS Station Mean Sea Level Pressure Differences for SWEX IOP #10**
## This notebook performs the following tasks:
> - #### Reads in station data downloaded from Synoptic for the Santa Barbara Aiport (KSBA), Santa Maria Aiport (KSMX), and Bakersfield Airport (KBFL). 
> - #### Pre-processes each station's data due to some non-conforming timesteps in each data file.
> - #### Subtracts the averaged mean sea level pressure (MSLP) at KSBA from the averaged MSLP at KSMX (KSBA-KSMX MSLP difference), and then performs the same computation for KSBA-KBFL (KSBA-KBFL MSLP difference).
> - #### Plots the results.

## **Import packages**
### Links to documentation for packages
> - #### [pathlib](https://docs.python.org/3/library/pathlib.html) | [numpy](https://numpy.org/doc/1.21/) | [pandas](https://pandas.pydata.org/pandas-docs/version/1.3.5/) | [xarray](https://docs.xarray.dev/en/stable/) | [matplotlib](https://matplotlib.org/3.5.3/index.html) 
> - #### Documentation for packages linked above should mostly correspond to the most stable versions, which may not be the exact versions used when creating this notebook.
> - #### Comments are also included in the actual code cells. Commented links above certain pieces of code are provided to help show readers/users of this code where I got some of my ideas/code from. This attempt to document my development of the code is not exhaustive, and may include links that are for extremely basic lines of code. It is possible that there may still be snippets of code here that I simply grabbed off the internet, from places like StackOverflow, without attributing any users or including a link.

In [ ]:
#-----------------------------------------------------
#Entire package imports
import pathlib
import numpy as np
import pandas as pd
import xarray as xr

#matplotlib
import matplotlib.patheffects
import matplotlib.pyplot as plt
#-----------------------------------------------------

## **Define variables that point to relative paths for relevant data**
### Data Information
> - #### **Automated Surface Observing System (ASOS)** 
>> - #### A set of stations that are placed at airports within the United States.
>> - #### Default File Format: csv files which contain observations from each station.
>> - #### Variables Used: 5-minute sea level pressure observations (default units: Pa; default Timezone: UTC)
>> - #### Sampling Information: See user's guide provided in the link below for more information.
>> - #### ASOS data was downloaded from the [Synoptic data website](https://synopticdata.com/) using the [SynopticPy package written by Brian Blaylock](https://github.com/blaylockbk/SynopticPy). The notebook used for downloading data from Synoptic can be found in ?. That notebook also contains more information on the conversion process used when converting the default csv files into NetCDF format. Please note that Synoptic provides the data in metric units and in the UTC timezone. If downloading from another source, please check the units.
>> - #### Some documentation for ASOS stations: [National Weather Service Description](https://www.weather.gov/asos/asostech) | [National Weather Service User's Guide](https://www.weather.gov/media/asos/aum-toc.pdf)
>> - #### Note: For reference, here is a breakdown of the currently used ASOS stations:
>>> - #### KBFL: Reports at 5-minute resolution beginning at the top of the hour
>>> - #### KSBA: Reports at 5-minute resolution beginning at the top of the hour
>>> - #### KSMX: Reports at 5-minute resolution beginning at the top of the hour
>>> - #### All sites have ~5-minute temporal resolution. However, a few non-conforming timesteps do show up in each station's timeseries.

In [ ]:
#-----------------------------------------------------
#Glob of all paths to station files
glob_ksba_file = sorted(pathlib.Path('./SWEX2022_datasets/ASOS/').glob('*KSBA*.nc'))
glob_ksmx_file = sorted(pathlib.Path('./SWEX2022_datasets/ASOS/').glob('*KSMX*.nc'))
glob_kbfl_file = sorted(pathlib.Path('./SWEX2022_datasets/ASOS/').glob('*KBFL*.nc'))

#Display glob paths to make sure we have files
display(glob_ksba_file)
display(glob_ksmx_file)
display(glob_kbfl_file)
#-----------------------------------------------------

## Calculate the MSLP difference between stations

#### **Notes**
> - #### Step #1: Open each station's data using xarray, subset timeseries data in time, and grab variable of interest (Mean Sea Level Pressure)
> - #### Step #2: Drop non-conforming timesteps in each station's timeseries using a mask. The identification of the non-conforming timesteps was completed and then removed for this notebook for clarity.
> - #### Step #3: Compute the KSBA-KSMX and KSBA-KBFL MSLP difference.

In [ ]:
#-----------------------------------------------------
#Open each station's data into an xarray Dataset
ds_ksba = xr.open_dataset(glob_ksba_file[0])
ds_ksmx = xr.open_dataset(glob_ksmx_file[0])
ds_kbfl = xr.open_dataset(glob_kbfl_file[0])

#Define start and end date strings to index data
start_date_str = '2022-05-12 17:00'
end_date_str   = '2022-05-13 17:00'

#Subset each dataset to cover just the IOP we are interested in
ds_ksba_subset = ds_ksba.sel(date_time=slice(start_date_str, end_date_str))
ds_ksmx_subset = ds_ksmx.sel(date_time=slice(start_date_str, end_date_str))
ds_kbfl_subset = ds_kbfl.sel(date_time=slice(start_date_str, end_date_str))

#Grab the variable we want (Mean Sea Level Pressure) from the subset data 
da_ksba_subset_mslp = ds_ksba_subset['sea_level_pressure_set_1d']
da_ksmx_subset_mslp = ds_ksmx_subset['sea_level_pressure_set_1d']
da_kbfl_subset_mslp = ds_kbfl_subset['sea_level_pressure_set_1d']

#Drop any timesteps that are not a multiple of 5-minutes (ASOS native resoltuion)
#We do this because sometimes there are pesky non-conforming timesteps which can make subtracting station data from one station with another incorrect.
da_ksba_subset_mslp_masked = da_ksba_subset_mslp.where(da_ksba_subset_mslp.date_time.dt.minute % 5 == 0, drop=True)
da_ksmx_subset_mslp_masked = da_ksmx_subset_mslp.where(da_ksmx_subset_mslp.date_time.dt.minute % 5 == 0, drop=True)
da_kbfl_subset_mslp_masked = da_kbfl_subset_mslp.where(da_ksmx_subset_mslp.date_time.dt.minute % 5 == 0, drop=True)
#-----------------------------------------------------
#Perform a quick check to ensure that the lenght of all masked station timeseries match and are equal to 288 (24 hours * 12 five minute timesteps per hour = 288)
if (da_ksba_subset_mslp_masked.date_time == da_ksmx_subset_mslp_masked.date_time).all() & (da_ksba_subset_mslp_masked.date_time == da_kbfl_subset_mslp_masked.date_time).all() == True:
    
    #Print a message stating the check passed
    print('All station timeseries are equal and have the correct amount of timesteps. Computing MSLP difference...')

#Else, do the following:    
else:
    
    #Raise an error with an informative message
    raise ValueError('Error: Not all stations have the same length. Please check to ensure computation are correct before computing the MSLP difference between stations.')
#-----------------------------------------------------
#Now compute the MSLP difference between stations
da_ksba_ksmx_mslp_diff = (da_ksba_subset_mslp_masked - da_ksmx_subset_mslp_masked).rename('ksba_ksmx_mslp_diff')
da_ksba_kbfl_mslp_diff = (da_ksba_subset_mslp_masked - da_kbfl_subset_mslp_masked).rename('ksba_kbfl_mslp_diff')
#-----------------------------------------------------

## **Plot MSLP differences for ASOS stations during SWEX IOP #10**
#### **Notes**
> - #### See in-line comments for any important information.

In [ ]:
#-----------------------------------------------------
#Set up plotting window
fig, ax = plt.subplots(figsize=(18,7))

#Define font cusomization items
fontdict_xtick_labels  = {'fontsize': 24, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}
fontdict_ytick_labels  = {'fontsize': 24, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}
fontdict_xaxis_labels  = {'fontsize': 24, 'fontweight': 'bold',   'fontname': 'Nimbus Roman'}
fontdict_yaxis_labels  = {'fontsize': 24, 'fontweight': 'bold',   'fontname': 'Nimbus Roman'}
fontdict_title_labels  = {'fontsize': 24, 'fontweight': 'bold',   'fontname': 'Nimbus Roman'}
fontdict_legend_labels = {'size': 20, 'weight': 'bold', 'family':'Nimbus Roman'}

#Plot MSLP differences
#Note that we divide by 100 to convert from units of Pa to hPa
ksba_ksmx_plot = ax.plot(da_ksba_ksmx_mslp_diff.date_time, 
                         da_ksba_ksmx_mslp_diff.values/100, 
                         color='blue', label='KSBA-KSMX', linewidth=5, zorder=3,
                         path_effects=[matplotlib.patheffects.Stroke(linewidth=7, foreground='black'), matplotlib.patheffects.Normal()])

ksba_kbfl_plot = ax.plot(da_ksba_kbfl_mslp_diff.date_time, 
                         da_ksba_kbfl_mslp_diff.values/100, 
                         color='red',  label='KSBA-KBFL', linewidth=5, zorder=3,
                         path_effects=[matplotlib.patheffects.Stroke(linewidth=7, foreground='black'), matplotlib.patheffects.Normal()])


#Add in lines to show MSLP difference thresholds for moderate to strong Sundowners for each pair of stations
#ksba_ksmx_threshold = ax.axhline(y=-5, color='blue', linestyle='--', label='KSBA-KSMX Threshold', linewidth=3)
#ksba_kbfl_threshold = ax.axhline(y=-7, color='red',  linestyle='--', label='KSBA-KBFL Threshold', linewidth=3)

#Colors used to delineate day and night in each plot
day_color   = 'white'
night_color = 'grey'

#Add in different vertical spans/lines for both axes
#Vertical span shows day/night times (in UTC) 
#Based on sunrise/sunset times taken from: https://www.timeanddate.com/sun/usa/santa-barbara?month=5&year=2022
ax.axvspan(pd.Timestamp(2022,5,12,12,57), pd.Timestamp(2022,5,13,2,52), facecolor=day_color, alpha=0.5,   label= 'Day')   #2022-05-12 sunset:  07:52PDT = 2022-05-13 02:52UTC
ax.axvspan(pd.Timestamp(2022,5,13,2,52), pd.Timestamp(2022,5,13,12,57), facecolor=night_color, alpha=0.5, label='Night')  #2022-05-13 sunrise: 05:57PDT = 2022-05-13 12:57UTC
ax.axvspan(pd.Timestamp(2022,5,13,12,57), pd.Timestamp(2022,5,14,2,57), facecolor=day_color, alpha=0.5)                   #2022-05-13 sunset:  19:52PDT = 2022-05-14 02:57UTC
#-----------------------------------------------------
#x-axis customizations
#Here we set the xticks/labels to be datetime objects and then do a bit of formatting to get the plot to look nice
#https://stackoverflow.com/questions/58113864/how-to-concatenate-datetime-arrays-without-changing-the-type

#Time xtick and xticklabels for all plots
xticks_utc = pd.date_range(start='2022-05-12 17:00', end='2022-05-13 17:00', freq='1H')
xticks_pdt = xticks_utc - np.timedelta64(7, 'h')
xticklabels  = [f'{str(xtick_pdt)[5:10]}\n{str(xtick_pdt)[10:13]} PDT' for xtick_pdt in xticks_pdt]

#Set x-axis items
ax.set_xlim([xticks_utc[0],xticks_utc[-1]])
ax.set_xlim([xticks_utc[0],xticks_utc[-1]])
ax.set_xticks(xticks_utc[::4])
ax.set_xticks(xticks_utc[::4])
ax.set_xticklabels(xticklabels[::4], **fontdict_xtick_labels)

#y-axis customizations
y_min, y_max, y_step = -8, 0, 1
yticks = np.arange(y_min,y_max+y_step, y_step)

#Set y-axis items
ax.set_ylim([y_min-1,y_max])
ax.set_yticks(yticks)
ax.set_yticks(yticks)
ax.set_yticklabels(yticks, **fontdict_ytick_labels)
ax.set_ylabel('MSLP Difference (hPa)', **fontdict_yaxis_labels)

#Plot a horizontal lines that spans the entire x-axis for each of the y_ticks we have
for y_tick in yticks:
    ax.axhline(y=y_tick, xmin=0, xmax=1, color='black', linestyle='--', linewidth=1, zorder=1)

#Plot a vertical line that spands the entire y-axis for every hour
for xtick_utc in xticks_utc:
    ax.axvline(x=xtick_utc, ymin=0, ymax=1, color='black', linestyle='--', linewidth=1, zorder=1)
    
#Make vertical line thicker where hours when we have a xticklabel
for xtick_utc in xticks_utc[::4]:
    ax.axvline(x=xtick_utc, ymin=0, ymax=1, color='black', linestyle='-', linewidth=2, zorder=1)
    
#General axis and legend customizations
ax.set_facecolor('white')
ax.tick_params(axis='both', which='major', length=15)
ax.set_title(f'b) 12-13 May 2022 ASOS MSLP Differences', **fontdict_title_labels)
ax.legend(loc='lower left', 
          ncol=2, fancybox=True, shadow=True, framealpha=1, 
          facecolor='snow', edgecolor='black', prop=fontdict_legend_labels)

#Small bit of nonsense to get a black edgecolor around all items in legend
for index, handle in enumerate(ax.legend_.legend_handles):
    if 'Line2' in str(handle):
        handle.set_linewidth(4)
        handle.set_path_effects([matplotlib.patheffects.Stroke(linewidth=6, foreground='black'), matplotlib.patheffects.Normal()])
    elif 'Rectangle' in str(handle):
        handle.set_edgecolor('black')
#-----------------------------------------------------
#Save figure    
plt.savefig('./figures/figure_02b_asos_mslp_differences.png', bbox_inches='tight', dpi=500)

#Show figure
plt.show()
#-----------------------------------------------------

## **Use "ImageMagick Command Line Tool to Append Figures 2a and 2b"**
#### **Notes**
> - #### Once we have generated figures 2a (seperate notebook) and 2b (this notebook), run the following command below to use the command line tool "ImageMagick" to append the two figures vertically. This produces a single final figure 2.

In [ ]:
#-----------------------------------------------------
#Run ImageMagick command
! convert ./figures/figure_02a_era5_synoptic_map.png ./figures/figure_02b_asos_mslp_differences.png -gravity center -append ./figures/figure_02_combined.png
#-----------------------------------------------------